In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path
from skimage.io import imread
from skimage.color import rgb2gray
from skimage.util import img_as_ubyte

In [ ]:
BLOCK_SIZE = 32

In [ ]:
def compute_ncc(a, b):
    a_mean = np.mean(a)
    b_mean = np.mean(b)
    a_dev = a - a_mean
    b_dev = b - b_mean
    numerator = np.sum(a_dev * b_dev)
    denominator = np.sqrt(np.sum(a_dev ** 2)) * np.sqrt(np.sum(b_dev ** 2))
    return numerator / (denominator + 1e-6)

def extract_ncc_features(ref_frame, tar_frame, block_size=32, search_range=1, step_size=1):
    h, w = ref_frame.shape
    features = []

    if block_size == 0:
        ncc_score = compute_ncc(ref_frame.flatten(), tar_frame.flatten())
        features.append({'block_y': 0, 'block_x': 0, 'ncc': ncc_score})
        return np.array([[ncc_score]])

    for y in range(0, h - block_size + 1, block_size):
        for x in range(0, w - block_size + 1, block_size):
            ref_patch = ref_frame[y:y+block_size, x:x+block_size].flatten()

            max_ncc = -1.0
            for dy in range(-search_range, search_range + 1, step_size):
                for dx in range(-search_range, search_range + 1, step_size):
                    ty = y + dy
                    tx = x + dx
                    ty = max(0, min(ty + block_size, h) - block_size)
                    tx = max(0, min(tx + block_size, w) - block_size)
                    tar_patch = tar_frame[ty:ty+block_size, tx:tx+block_size].flatten()
                    ncc_score = compute_ncc(ref_patch, tar_patch)
                    max_ncc = max(max_ncc, ncc_score)

            print(f"NCC at block (y={y}, x={x}) = {max_ncc:.4f}")
            features.append(max_ncc)

    return np.array(features).reshape(h // block_size, w // block_size)

In [ ]:
frame_dir = Path("dataset/DAVIS/curling")
frame_paths = sorted(frame_dir.glob("*.jpg"))
gray_frames = []

for path in frame_paths:
    image = imread(path)
    gray = rgb2gray(image) if image.ndim == 3 else image
    gray = img_as_ubyte(gray)
    gray_frames.append(gray)

ref_frame, tar_frame = gray_frames[0], gray_frames[1]
ncc_map = extract_ncc_features(ref_frame, tar_frame, BLOCK_SIZE)

plt.figure(figsize=(6, 5))
plt.imshow(ncc_map, cmap='hot', interpolation='nearest')
plt.title(f"NCC Heatmap (block_size={BLOCK_SIZE})")
plt.colorbar()
plt.axis("off")
plt.tight_layout()
plt.show()